# Training FDFL

In [122]:
import sys
import warnings
import time
import copy
import json
from datetime import datetime
from itertools import product

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.optim as optim
import torch.nn.functional as F
from torch import nn
from torch.autograd import Function
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import cvxpy as cp

from pyepo.model.opt import optModel
sys.path.insert(0, 'E:\\User\\Stevens\\MyRepo\\FDFL\\helper')
sys.path.insert(0, 'E:\\User\\Stevens\\MyRepo\\fold-opt-package\\fold_opt')

from myutil import *
from features import get_all_features

# Suppress warnings
warnings.filterwarnings("ignore")
from GMRES import *
from fold_opt import *

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
print(device)


cpu


### Import and Process Data

Running with simpler model
Alpha = 0.5, 1.5, 2, 3

In [123]:
alpha, Q = 2, 1000

In [124]:
df = pd.read_csv('data/data.csv')
# fix random seed for reproducibility

# report statistics on this dataset
df = df.sample(n=5000, random_state=1)

columns_to_keep = [
    'risk_score_t', 'program_enrolled_t', 'cost_t', 'cost_avoidable_t', 'race', 'dem_female', 'gagne_sum_tm1', 'gagne_sum_t', 
    'risk_score_percentile', 'screening_eligible', 'avoidable_cost_mapped', 'propensity_score', 'g_binary', 
    'g_continuous', 'utility_binary', 'utility_continuous'
]
# for race 0 is white, 1 is black
df_stat = df[columns_to_keep]
df_feature = df[[col for col in df.columns if col not in columns_to_keep]]

# Replace all values less than 0.1 with 0.1
#df['risk_score_t'] = df['risk_score_t'].apply(lambda x: 0.1 if x < 0.1 else x)
df['g_continuous'] = df['g_continuous'].apply(lambda x: 0.1 if x < 0.1 else x)


risk = df['risk_score_t'].values
risk = risk + 0.001 if 0 in risk else risk


feats = df[get_all_features(df)].values
gainF = df['g_continuous'].values
decision = df['propensity_score'].values
cost = np.random.normal(1, 0.5, len(risk)).clip(0.1, 2)
race = df['race'].values

# transform the features
scaler = StandardScaler()
feats = scaler.fit_transform(feats)

from sklearn.model_selection import train_test_split



In [125]:
class optDataset(Dataset):
    def __init__(self, optmodel, feats, risk, gainF, cost, race, alpha=alpha, Q=Q):
        self.feats = torch.from_numpy(feats).float()
        self.risk = torch.from_numpy(risk).float()
        self.gainF = torch.from_numpy(gainF).float()
        self.cost = torch.from_numpy(cost).float()
        self.race = torch.from_numpy(race).float()
        self.optmodel = optmodel

        # Solve for w*, z* using separate risk and gainF # Ensure a separate instance
        self.w_star, self.z_star = self.optmodel(self.risk, self.gainF, self.cost, alpha=alpha, Q=Q)

        self.w_star = torch.tensor(self.w_star, dtype=torch.float)
        self.z_star = torch.tensor(self.z_star, dtype=torch.float)

    def __len__(self):
        return len(self.feats)

    def __getitem__(self, idx):
        return self.feats, self.risk, self.gainF, self.cost, self.race, self.w_star, self.z_star


In [126]:
# class FairRiskPredictor(nn.Module):
#     def __init__(self, input_dim, dropout_rate=0.1):
#         super().__init__()
#         self.model = nn.Sequential(
#             nn.Linear(input_dim, 1),
#             nn.Softplus()
#         )
            
                    
#     def forward(self, x):
#         return self.model(x).squeeze(-1)
    
class FairRiskPredictor(nn.Module):
    def __init__(self, input_dim, dropout_rate=0.1):
        super().__init__()
        self.model = nn.Sequential(
            # First layer with batch normalization
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            
            # Output layer
            nn.Linear(64, 1),
            nn.Softplus()
        )
            
    def forward(self, x):
        return self.model(x).squeeze(-1)

In [127]:
def regret(predmodel, optmodel, dataloader, alpha=alpha, Q=Q):
    predmodel.eval()
    feats, risk, gainF, cost, race, opt_sol, opt_val = next(iter(dataloader))

    if torch.cuda.is_available():
        feats, risk, gainF, cost, race, opt_sol, opt_val = feats.cuda(), risk.cuda(), gainF.cuda(), cost.cuda(), race.cuda(), opt_sol.cuda(), opt_val.cuda()

    with torch.no_grad():
        pred_risk = predmodel(feats)

    risk = risk.detach().to('cpu').numpy()
    pred_risk = pred_risk.detach().to('cpu').numpy().flatten()
    pred_risk = pred_risk.clip(min=0.001)
    gainF = gainF.detach().to('cpu').numpy().flatten()
    cost = cost.detach().to('cpu').numpy().flatten()
    pred_sol, _ = optmodel(gainF, pred_risk, cost, alpha, Q)
        
    pred_obj = AlphaFairness(gainF * risk * pred_sol, alpha)

    normalized_regret = (opt_val - pred_obj) / (abs(opt_val) + 1e-7)
    predmodel.train()
    return normalized_regret

# Helper Functions

In [128]:
# Setup training parameters

optmodel = solve_closed_form

# Perform train-test split
feats_train, feats_test, gainF_train, gainF_test, risk_train, risk_test, cost_train, cost_test, race_train, race_test = train_test_split(
    feats, gainF, risk, cost, df['race'].values, test_size=0.5, random_state=2
)

print(f"Train size: {feats_train.shape[0]}")
print(f"Test size: {feats_test.shape[0]}")

dataset_train = optDataset(optmodel, feats_train, risk_train, gainF_train, cost_train, race_train, alpha=alpha, Q=Q)
dataset_test = optDataset(optmodel, feats_test, risk_test, gainF_test, cost_test, race_test, alpha=alpha, Q=Q)

# Create dataloaders
dataloader_train = DataLoader(dataset_train, batch_size=1, shuffle=False)
dataloader_test = DataLoader(dataset_test, batch_size=1, shuffle=False)

predmodel = FairRiskPredictor(feats_train.shape[1])
predmodel.to(device)
# save the initial model
# torch.save(predmodel.state_dict(), 'initial_model.pth')
# load the initial model

Train size: 2500
Test size: 2500


FairRiskPredictor(
  (model): Sequential(
    (0): Linear(in_features=149, out_features=64, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=64, out_features=1, bias=True)
    (4): Softplus(beta=1, threshold=20)
  )
)

In [129]:
import time
import torch
import torch.nn as nn

# assume make_foldopt_layer and alpha_fair are already in scope from your fold-opt code
# from fold_opt import make_foldopt_layer, alpha_fair

def trainFairModelFoldOpt(
    predmodel,
    loader_train,
    loader_test,
    alpha,
    Q,
    lambda_fairness=0.1,
    num_epochs=10,
    lr_pred=1e-3,
    weight_decay=1e-4,
    pgd_lr=1e-2,
    n_fixedpt=200,
    backprop_rule="GMRES"
):
    device = next(predmodel.parameters()).device
    optimizer = torch.optim.Adam(predmodel.parameters(), lr=lr_pred, weight_decay=weight_decay)

    logs = {
        "train_loss": [],
        "train_mse": [],
        "train_fair": [],
        "train_regret": []
    }

    predmodel.train()
    for epoch in range(1, num_epochs + 1):
        t0 = time.time()

        # --- pull the one-and-only batch (the entire dataset) ---
        feats, risk, gainF, cost, race, opt_d, opt_obj = next(iter(loader_train))

        # squeeze away the dummy batch‐dim and move to device
        feats   = feats.squeeze(0).to(device)   # (n,m)
        risk    = risk.squeeze(0).to(device)    # (n)
        gainF   = gainF.squeeze(0).to(device)   # (n)
        cost    = cost.squeeze(0).to(device)    # (n)
        race    = race.squeeze(0).to(device)    # (n)
        opt_d   = opt_d.squeeze(0).to(device)   # (n)
        opt_obj = opt_obj.squeeze(0).to(device) # scalar or (1)

        # --- forward pass: predict risk ---
        pred_risk = predmodel(feats).clamp(min=1e-3)  # (n,)

        # --- build a Fold-Opt layer for this batch ---
        #    it will map r_batch (1,n) → d_pred (1,n)
        fold_layer = make_foldopt_layer(
            gainF, cost, alpha, Q,
            lr=pgd_lr,
            n_fixedpt=n_fixedpt,
            rule=backprop_rule
        )

        # run it (we need a 2-d input for the layer)
        d_pred = fold_layer(pred_risk.unsqueeze(0)).squeeze(0)  # (n,)

        # --- compute regret loss via alpha‐fairness ---
        u_pred    = d_pred * risk * gainF               # (n,)
        pred_obj  = alpha_fair(u_pred.unsqueeze(0), alpha)   # (1,)
        regret_l1 = (opt_obj - pred_obj) / (opt_obj.abs() + 1e-7)  # (1,)

        # --- fairness penalty: difference in MSE across race groups ---
        m0 = (pred_risk[race == 0] - risk[race == 0]).pow(2).mean() if (race==0).any() else torch.tensor(0., device=device)
        m1 = (pred_risk[race == 1] - risk[race == 1]).pow(2).mean() if (race==1).any() else torch.tensor(0., device=device)
        fair_reg = torch.abs(m0 - m1)

        # --- total loss & backward ---
        loss = regret_l1 + lambda_fairness * fair_reg
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # --- compute simple MSE for logging ---
        mse_train = (pred_risk - risk).pow(2).mean()

        # --- log everything ---
        logs["train_loss"].append(loss.item())
        logs["train_mse"].append(mse_train.item())
        logs["train_fair"].append(fair_reg.item())
        logs["train_regret"].append(regret_l1.item())

        # (optional) print progress
        print(f"Epoch {epoch:2d} | Loss={loss.item():.4f} | MSE={mse_train.item():.4f} | Fair={fair_reg.item():.4f} | Regret={regret_l1.item():.4f} | {time.time()-t0:.1f}s")

    predmodel.eval()
    return logs


In [130]:
logs = trainFairModelFoldOpt(
    predmodel,
    dataloader_train,
    dataloader_test,
    alpha=1.50,
    Q=1000,
    lambda_fairness=0,
    num_epochs=50,
    lr_pred=1e-3
)


Epoch  1 | Loss=-0.0475 | MSE=48.4381 | Fair=65.9715 | Regret=-0.0475 | 1.0s
Epoch  2 | Loss=-0.0502 | MSE=48.5404 | Fair=65.7493 | Regret=-0.0502 | 1.0s
Epoch  3 | Loss=-0.0538 | MSE=48.5815 | Fair=65.7072 | Regret=-0.0538 | 1.0s
Epoch  4 | Loss=-0.0576 | MSE=48.6448 | Fair=65.7161 | Regret=-0.0576 | 1.0s
Epoch  5 | Loss=-0.0591 | MSE=48.7762 | Fair=65.8527 | Regret=-0.0591 | 1.0s
Epoch  6 | Loss=-0.0633 | MSE=48.8298 | Fair=65.6968 | Regret=-0.0633 | 1.0s
Epoch  7 | Loss=-0.0664 | MSE=48.9592 | Fair=65.9307 | Regret=-0.0664 | 1.0s
Epoch  8 | Loss=-0.0693 | MSE=49.0397 | Fair=66.1033 | Regret=-0.0693 | 1.0s
Epoch  9 | Loss=-0.0736 | MSE=49.1447 | Fair=66.0042 | Regret=-0.0736 | 1.0s
Epoch 10 | Loss=-0.0775 | MSE=49.2398 | Fair=66.0903 | Regret=-0.0775 | 1.0s


KeyboardInterrupt: 

In [ ]:
predmodel = FairRiskPredictor(feats_train.shape[1])
predmodel.to(device)

logs = trainFairModelFoldOpt(
    predmodel,
    dataloader_train,
    dataloader_test,
    alpha=1.9,
    Q=1000,
    lambda_fairness=0,
    num_epochs=20,
    lr_pred=1e-3
)


Epoch  1 | Loss=-0.1018 | MSE=48.7463 | Fair=66.3695 | Regret=-0.1018 | 1.0s
Epoch  2 | Loss=-0.1058 | MSE=48.7710 | Fair=66.2251 | Regret=-0.1058 | 1.0s
Epoch  3 | Loss=-0.1114 | MSE=48.8249 | Fair=66.2088 | Regret=-0.1114 | 1.0s
Epoch  4 | Loss=-0.1164 | MSE=48.8049 | Fair=66.0563 | Regret=-0.1164 | 1.0s
Epoch  5 | Loss=-0.1212 | MSE=48.8947 | Fair=65.9276 | Regret=-0.1212 | 1.0s
Epoch  6 | Loss=-0.1261 | MSE=48.9604 | Fair=66.0105 | Regret=-0.1261 | 1.0s
Epoch  7 | Loss=-0.1285 | MSE=49.0425 | Fair=65.8967 | Regret=-0.1285 | 1.0s
Epoch  8 | Loss=-0.1342 | MSE=49.0996 | Fair=66.0029 | Regret=-0.1342 | 1.0s
Epoch  9 | Loss=-0.1361 | MSE=49.2121 | Fair=65.9900 | Regret=-0.1361 | 1.0s
Epoch 10 | Loss=-0.1361 | MSE=49.3038 | Fair=66.0044 | Regret=-0.1361 | 1.0s
Epoch 11 | Loss=-0.1423 | MSE=49.4695 | Fair=66.1590 | Regret=-0.1423 | 1.0s
Epoch 12 | Loss=-0.1435 | MSE=49.5863 | Fair=66.1687 | Regret=-0.1435 | 1.0s
Epoch 13 | Loss=-0.1460 | MSE=49.7538 | Fair=66.5696 | Regret=-0.1460 | 1.0s

In [ ]:
predmodel = FairRiskPredictor(feats_train.shape[1])
predmodel.to(device)

logs = trainFairModelFoldOpt(
    predmodel,
    dataloader_train,
    dataloader_test,
    alpha=0.5,
    Q=1000,
    lambda_fairness=0,
    num_epochs=20,
    lr_pred=1e-3
)


Epoch  1 | Loss=-2.6334 | MSE=48.2352 | Fair=66.4849 | Regret=-2.6334 | 1.7s
Epoch  2 | Loss=-2.6274 | MSE=48.2610 | Fair=66.6189 | Regret=-2.6274 | 1.7s
Epoch  3 | Loss=-2.6347 | MSE=48.2671 | Fair=66.8725 | Regret=-2.6347 | 1.7s
Epoch  4 | Loss=-2.6298 | MSE=48.2601 | Fair=66.6652 | Regret=-2.6298 | 1.7s
Epoch  5 | Loss=-2.6319 | MSE=48.2638 | Fair=66.6183 | Regret=-2.6319 | 1.7s
Epoch  6 | Loss=-2.6304 | MSE=48.1750 | Fair=66.3980 | Regret=-2.6304 | 1.7s
Epoch  7 | Loss=-2.6325 | MSE=48.1621 | Fair=66.3467 | Regret=-2.6325 | 1.7s
Epoch  8 | Loss=-2.6333 | MSE=48.1063 | Fair=66.1515 | Regret=-2.6333 | 1.7s
Epoch  9 | Loss=-2.6428 | MSE=48.0501 | Fair=66.1426 | Regret=-2.6428 | 1.0s
Epoch 10 | Loss=-2.6408 | MSE=48.0377 | Fair=65.7064 | Regret=-2.6408 | 1.0s
Epoch 11 | Loss=-2.6386 | MSE=48.0283 | Fair=66.0297 | Regret=-2.6386 | 1.0s
Epoch 12 | Loss=-2.6440 | MSE=47.9888 | Fair=65.8849 | Regret=-2.6440 | 1.0s
Epoch 13 | Loss=-2.6450 | MSE=47.9276 | Fair=65.2172 | Regret=-2.6450 | 1.0s

In [ ]:
# E:\User\Stevens\MyRepo\FDFL\res\cvxpylayer\LR\results_cvx_0p5.pkl
# E:\User\Stevens\MyRepo\FDFL\res\cvxpylayer\LR\results_cvx_0p5_fair.pkl
# E:\User\Stevens\MyRepo\FDFL\res\cvxpylayer\LR\results_cvx_1p5.pkl
# E:\User\Stevens\MyRepo\FDFL\res\cvxpylayer\LR\results_cvx_1p5_fair.pkl
# E:\User\Stevens\MyRepo\FDFL\res\cvxpylayer\LR\results_cvx_2.pkl
# E:\User\Stevens\MyRepo\FDFL\res\cvxpylayer\LR\results_cvx_2_fair.pkl
# E:\User\Stevens\MyRepo\FDFL\res\cvxpylayer\LR\results_cvx_inf.pkl
# E:\User\Stevens\MyRepo\FDFL\res\cvxpylayer\LR\results_cvx_inf_fair.pkl